In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.modeling import models
from astropy import units as u
from astropy.constants import c,h

#Convert c and h to cgs
c = c.cgs
h = h.cgs

#1.)

##Source Object

In [ ]:
class Source:

  def __init__(self,name):
    """Instantiation of the Source.

    This function initializes the Source class and all parameters concerning it

    Ex:
        Source("Vega")

    Args:
        None

    """
    self.name = name

    #Need to set object's SED and magnitude here. Not sure how.



  def photon_flux(self,type, T, lam):
    """ Determines photon flux at given wavelength of the source

    Takes type (e.g. blackbody), temperature (if blackbody), and wavelength and
    returns flux

    Parameters
    ----------
    type: String for source type ("blackbody" or "###")
    T: Temperature in K for use ONLY when type is "blackbody"
    lam: wavelength in angstroms

    Returns
    -------
    f_photon: Photon flux
    """

    #Attach units of angstroms to lam
    lam = lam * u.AA
    nu = c/lam

    if(type == "Blackbody"):
      F = models.BlackBody(T*u.K)
      f_lam = F(lam)*(nu**2)/(c*1e8*u.AA/u.cm) #Convert from 1/Hz => 1/AA
    else:
      f_lam = 0

    #Convert to photon flux
    f_photon = f_lam*lam/(h*c)
    #NEED TO ASK: Normalize to brightness? Does the photon flux over all wavelengths
    #need to integrate to 1??

    return(f_photon)

#2.)

##Atmosphere Object

In [ ]:
class Atmosphere:

  def __init__(self,name):
    """Instantiation of the Atmosphere.

    This function initializes the Atmosphere class and all parameters concerning it

    Args:
        None

    """
    self.name = name
    """
    if (name=="constant"):
      a_lam = 0.8
    #NEED TO ASK: How to define parameters in the __init__ function
  """

  def transmission(self, lam):
      """ Determines atmospheric transmission at given wavelength of the source

      Takes wavelength and returns tranmission coefficient

      Parameters
      ----------
      lam: wavelength in angstroms

      Returns
      -------
      a_lam: Atmospheric tranmission function value [0,1]
      """

      #Assume constant for now
      a_lam = 0.8

      return(a_lam)


#3.)

##Telescope Object

In [ ]:
class Telescope:

  def __init__(self,name):
    """Instantiation of the Telescope.

    This function initializes the Telescope class and all parameters concerning it

    Ex:
        Source("TMO")

    Args:
        None

    """
    self.name = name

  def area(self, diam):
      """ Determines telescope objective area

      Takes diameter and returns objective area

      Parameters
      ----------
      diam: diameter in cm

      Returns
      -------
      area: objective area in cm^2
      """

      return(np.pi*(1/4)*(diam)**2)

  def throughput(self, name, lam):
      """ Determines telescope throughput

      Takes telescope name and wavelength and returns throughput

      Parameters
      ----------
      name: string of telescope name
      lam: wavelength in angstroms

      Returns
      -------
      m_lam: telescope throughput at lambda
      """

      if(name == "TMO"):
        m_lam = 0.5**(1/3)

      return(m_lam)


########################### DETECTOR CLASS #####################################

class Detector:

  def __init__(self,name):
    """Instantiation of the Detector.

    This function initializes the Detector class and all parameters concerning it

    Ex:
        Source("CCD")

    Args:
        None

    """
    self.name = name

  def response(self, lam):
      """ Determines detectore response

      Takes wavelength and returns detector response

      Parameters
      ----------
      lam: wavelength in angstroms

      Returns
      -------
      d_lam: detector response
      """

      d_lam = 0.5**(1/3)

      return(d_lam)

#4.)

##Instrument Object

In [ ]:
class Instrument:

  def __init__(self,name):
    """Instantiation of the Instrument.

    This function initializes the Instrument class and all parameters concerning it

    Ex:
        Source("TMO")

    Args:
        None

    """
    self.name = name

  def transmission(self, lam):
      """ Determines instrument transmission

      Takes wavelength and returns instrument transmission

      Parameters
      ----------
      lam: wavelength in angstroms

      Returns
      -------
      i_lam: instrument transmission
      """

      i_lam = 0.5**(1/3)

      return(i_lam)

#5.)

##Filter Object

In [ ]:
class Filter:

  def __init__(self,name):
    """Instantiation of the Filter.

    This function initializes the Filter class and all parameters concerning it

    Ex:
        Source("V")

    Args:
        None

    """
    self.name = name

  def response(self, band, lam):
      """ Determines filter response

      Takes wavelength and returns filter response

      Parameters
      ----------
      band: string indicating band pass
      lam:  wavelength in angstroms

      Returns
      -------
      f_lam: filter response
      """

      i_lam = 0

      if(band=="V"):
        lower = 5500-850/2
        higher = 5500+850/2
        if(lam>=lower and lam<=higher):
          i_lam = 0.8


      return(i_lam)

#6.)

##Signal Calculation

In [ ]:
#Create range of wavelengths
lam = np.arange(3000,7500,1)

#Initialize signal
S = 0

#Initialize the classes
src = Source("Vega")
atm = Atmosphere("earth")
tel = Telescope("TMO")
ins = Instrument("TMO")
det = Detector("CCD")
fil = Filter("V")

#Integrate over wavelengths
#for i in range(len(lam)):
F_lam = src.photon_flux("Blackbody", 6000 ,lam)
a_lam = atm.transmission(lam)
m_lam = tel.throughput("TMO",lam)
i_lam = ins.transmission(lam)

f_lam = []
for i in range(len(lam)):
  f_lam.append(fil.response("V",lam[i]))

d_lam = det.response(lam)
T = tel.area(60)

S = np.sum(T*F_lam*a_lam*m_lam*i_lam*f_lam*d_lam)

# Exposure Calculator II

In [ ]:
class Signal:

  def __init__(self,name):
    """Instantiation of the Signal being collected.

    This function initializes the Signal class and all parameters concerning it

    Ex:
        Source("photometer")

    Args:
        None

    """
    self.name = name

  def SNR(self,type,lam,T,F_lam,a_lam,m_lam,i_lam,f_lam,d_lam,t,sig_rn,Npix):
      """ Determines signal to noise ratio

      Takes type of detector, throughput/transmission terms, wavelenghts, and
      exposure time and returns SNR

      Parameters
      ----------
      type: type of detector (e.g. photometer or spectrograph)
      lam:  wavelength in angstroms
      T: telescope area
      F_lam: photon flux per wavelength
      a_lam: atmospheric transmission per wavelength
      m_lam: mirror throughput per wavelength
      i_lam: instrumental throughput per wavelength
      f_lam: filter throughput per wavlength
      d_lam: detector response per wavlength
      t: exposure time
      sig_rn: readout noise per pixel
      Npix: number of pixels from aperture size

      Returns
      -------
      SNR: signal to noise ratio (either single value or array)
      """

      if (type == "photometer"):
        #Compute signal
        S = np.sum(T*t*F_lam*a_lam*m_lam*i_lam*f_lam*d_lam)

      elif (type == "spectrograph"):
        #Compute individual signals
        S = T*t*F_lam*a_lam*m_lam*i_lam*f_lam*d_lam

      else:
        S = 0

      #Compute SNR
      SNR = S/np.sqrt(S + Npix*sig_rn**2)


      return(SNR)

def exp_time(self,type,lam,T,F_lam,a_lam,m_lam,i_lam,f_lam,d_lam,SNR,sig_rn,Npix):
      """ Determines exposure time ratio

      Takes type of detector, throughput/transmission terms, wavelengths, and
      SNR and returns exposure time

      Parameters
      ----------
      type: type of detector (e.g. photometer or spectrograph)
      lam:  wavelength in angstroms
      T: telescope area
      F_lam: photon flux per wavelength
      a_lam: atmospheric transmission per wavelength
      m_lam: mirror throughput per wavelength
      i_lam: instrumental throughput per wavelength
      f_lam: filter throughput per wavlength
      d_lam: detector response per wavlength
      SNR: signal to noise ratio (either single value or array)
      sig_rn: readout noise per pixel
      Npix: number of pixels from aperture size

      Returns
      -------
      t: exposure time
      """

      #Solve for signal using quadratic formula
      a = SNR**-2
      b = -1
      c = -Npix*sig_rn**2
      sol = (-b + np.sqrt(b**2 - 4*a*c))/(2*a) #Always take positive value

      #Compute time from this
      t = S/(T*F_lam*a_lam*m_lam*i_lam*f_lam*d_lam)

      return(t)


<Quantity 1.51492899e+15 Angstrom erg / (Hz s sr cm2)>